# Ranked Candidates

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

data_Recruiter = pd.read_csv("./job_post.csv")
data_Recruiter

,Company Name,Position,Skills,Experience,10th,12th,Engineering
0,ABC,Data Science,"Python, Machine Learning, Pandas, Numpy",0,60,60,72
1,WEB,Web Designing,"Bootstrap,Javascript,HTML5,Jquery,CSS",1,60,65,60
2,BLC,Blockchain,"Blockchain, Ethereum",1,70,65,70
3,JAVA,Java Developer,"Java,Hibernate,Springboot,Eclipse",0,60,60,60
4,SAP,SAP Developer,"Java,Hibernate,Springboot,Analytics,Testing",2,60,60,60
5,AUTO,Automation Testing,"Selenium,Data Aquistion,API,Testing",1,70,75,70
6,PYTHON,Python Developer,"Python,Mysql,numpy,Django",0,60,65,60
7,DEVOPS,DevOps Engineer,"Devops,Testing,Mysql,Java",0,60,60,60
8,NETWORK,Network Security Engineer,"Router,Network Security,Monitoring,Python",1,65,60,70


In [2]:
data_Candidate = pd.read_csv("./candidate_data.csv")
data_Candidate

,Name,Position,Skills,Experience,10th,12th,Engineering
0,Kunal,Data Science,"Python, Machine Learning",0,60,60,72
1,Omkar,Data Science,"Python, Machine Learning, Pandas, Numpy",1,70,65,70
2,Nilesh,Python Developer,"Python, Machine Learning, Pandas, Django",0,70,70,75
3,Raj,Web Designing,"HTML5,CSS,Javascript,Java,Mysql",2,75,75,75
4,Jake,Automation Testing,"Testing,Data Acquistion,Python",1,70,75,70
5,John,Java Developer,"Java, Eclipse,Hibernate,Mysql",0,65,65,70


In [3]:
one_Recruiter = data_Recruiter.loc[data_Recruiter['Company Name'] == 'ABC']
one_Recruiter

,Company Name,Position,Skills,Experience,10th,12th,Engineering
0,ABC,Data Science,"Python, Machine Learning, Pandas, Numpy",0,60,60,72


In [4]:
matched_candidates = pd.DataFrame()
for index,row in data_Candidate.iterrows():
    for index2,row2 in one_Recruiter.iterrows():
        if row['Experience'] >= row2['Experience'] and row['Engineering'] >= row2['Engineering'] and row['10th'] >= row2['10th'] and row['12th'] >= row2['12th']:
            matched_candidates = pd.concat([matched_candidates,pd.DataFrame(row).T],ignore_index=True)
matched_candidates

,Name,Position,Skills,Experience,10th,12th,Engineering
0,Kunal,Data Science,"Python, Machine Learning",0,60,60,72
1,Nilesh,Python Developer,"Python, Machine Learning, Pandas, Django",0,70,70,75
2,Raj,Web Designing,"HTML5,CSS,Javascript,Java,Mysql",2,75,75,75


## Candidate

In [5]:
matched_candidates['text'] = matched_candidates['Position'].map(str) + " " +matched_candidates['Skills']
matched_candidates

,Name,Position,Skills,Experience,10th,12th,Engineering,text
0,Kunal,Data Science,"Python, Machine Learning",0,60,60,72,"Data Science Python, Machine Learning"
1,Nilesh,Python Developer,"Python, Machine Learning, Pandas, Django",0,70,70,75,"Python Developer Python, Machine Learning, Pan..."
2,Raj,Web Designing,"HTML5,CSS,Javascript,Java,Mysql",2,75,75,75,"Web Designing HTML5,CSS,Javascript,Java,Mysql"


In [6]:
df_C = matched_candidates[['Name', 'text','Experience','10th','12th','Engineering']]
df_C = df_C.fillna(" ")
df_C

,Name,text,Experience,10th,12th,Engineering
0,Kunal,"Data Science Python, Machine Learning",0,60,60,72
1,Nilesh,"Python Developer Python, Machine Learning, Pan...",0,70,70,75
2,Raj,"Web Designing HTML5,CSS,Javascript,Java,Mysql",2,75,75,75


## Function to Clean text


In [7]:
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "",text)
  text=re.sub("(\\d|\\W)+"," ",text) 
  text = text.replace("nbsp", "")
  clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
  clean_text2 = [word for word in clean_text if black_txt(word)]
  return " ".join(clean_text2)

In [8]:
df_C['text'] = df_C['text'].apply(clean_txt)
df_C

,Name,text,Experience,10th,12th,Engineering
0,Kunal,data science python machine learn,0,60,60,72
1,Nilesh,python developer python machine learn pandas d...,0,70,70,75
2,Raj,web design html css javascript java mysql,2,75,75,75


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_candidates = tfidf_vectorizer.fit_transform((df_C['text']))
tfidf_candidates

<3x15 sparse matrix of type '<class 'numpy.float64'>'
	with 18 stored elements in Compressed Sparse Row format>

## Recruiter

In [10]:
one_Recruiter['text'] = one_Recruiter['Position'].map(str) + " " + one_Recruiter['Skills']
one_Recruiter

,Company Name,Position,Skills,Experience,10th,12th,Engineering,text
0,ABC,Data Science,"Python, Machine Learning, Pandas, Numpy",0,60,60,72,"Data Science Python, Machine Learning, Pandas,..."


In [11]:
df_R = one_Recruiter[['Company Name', 'text']]
df_R = df_R.fillna(" ")
df_R

,Company Name,text
0,ABC,"Data Science Python, Machine Learning, Pandas,..."


In [12]:
df_R['text'] = df_R['text'].apply(clean_txt)
df_R

,Company Name,text
0,ABC,data science python machine learn pandas numpy


In [13]:
from sklearn.metrics.pairwise import cosine_similarity
tfidf_recruiter = tfidf_vectorizer.transform(df_R['text'])
cos_similarity_tfidf = map(lambda x: cosine_similarity(tfidf_recruiter, x),tfidf_candidates)

In [14]:
output2 = list(cos_similarity_tfidf)

In [15]:
def ranked_candidates(top, df_C, scores):
    ranked_candidates = pd.DataFrame(columns = ['Name', 'score'])
    count = 0
    for i in top:
        ranked_candidates.at[count, 'Name'] = df_C['Name'][i]
        exp = df_C['Experience'][i]*100*0.10
        edu = (df_C['10th'][i]+df_C['12th'][i]+df_C['Engineering'][i])/3*0.30
        score = (scores[count]*100*0.60)+edu+exp
        ranked_candidates.at[count, 'score'] =  score
        count += 1
    return ranked_candidates

In [16]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:10]
list_scores = [output2[i][0][0] for i in top]
ranked_candidates = ranked_candidates(top,df_C, list_scores)
ranked_candidates

,Name,score
0,Kunal,72.489175
1,Nilesh,57.41836
2,Raj,42.5
